In [1]:
from imutils import face_utils
import dlib
import cv2
import imutils
import pyautogui
import numpy as np
import import_ipynb
from utils import *
from scipy.spatial import distance as dist
import time
#from pynput.mouse import Button, Controller

importing Jupyter notebook from utils.ipynb


In [2]:
WHITE_COLOR = (255, 255, 255)
YELLOW_COLOR = (0, 255, 255)
RED_COLOR = (0, 0, 255)
GREEN_COLOR = (0, 255, 0)
BLUE_COLOR = (255, 0, 0)
BLACK_COLOR = (0, 0, 0)
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 4
MOUTH_AR_THRESH = 0.6
MOUTH_AR_CONSECUTIVE_FRAMES = 3
SCROLL_MODE = False
SMILE_THRESH = 140
SMILE_CONSECUTIVE_FRAMES = 5
# initialize the frame counters and the total number of blinks
EYE_COUNTER = 0
MOUTH_COUNTER = 0
SMILE_COUNTER = 0

In [3]:
pyautogui.FAILSAFE = False

In [4]:
p = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

In [5]:
# Grab the indexes of the facial landmarks for the left and right eye, nose and mouth respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(nStart, nEnd) = face_utils.FACIAL_LANDMARKS_IDXS["nose"]
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

In [6]:
cap = cv2.VideoCapture(0)

#Initially set cursor to centre of the screen  
#For mac screen I am using the resolution is 1439*899, so we take centre as 720*450, 
#Needs to changed according to the screen we are using
pyautogui.moveTo(720,450)

isFirstFrame = True
prevNoseCoords = (0,0)

while True:
    # Getting out image by webcam 
    time1 = time.time()
    _, image = cap.read()
    image = imutils.resize(image, width=1439, height=899)
    # Converting the image to gray scale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
    # Get faces into webcam's image
    rects = detector(gray, 0)
    
    # Restricted to single face because multiple faces will create problem
    if len(rects) > 0:
        rect = rects[0]
    else:
        #cv2.imshow("Frame", image)
        key = cv2.waitKey(1) & 0xFF
        continue

        # Make the prediction and transfom it to numpy array
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)
        
    mouth = shape[mStart:mEnd]
    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]
    nose = shape[nStart:nEnd]
    
    # Because I flipped the frame, left is right, right is left.
    temp = leftEye
    leftEye = rightEye
    rightEye = temp
    
    mar = mouth_aspect_ratio(mouth)
    leftEAR = eye_aspect_ratio(leftEye)
    rightEAR = eye_aspect_ratio(rightEye)
    ear = (leftEAR + rightEAR) / 2.0
    diff_ear = np.abs(leftEAR - rightEAR)
    smile_dist = np.linalg.norm(mouth[0]-mouth[6])
    #print(smile_dist)
    #print(leftEAR, " ", rightEAR)
    
    #Moving the mouse pointer
    nose_point = (nose[3, 0], nose[3, 1])
    if(isFirstFrame):
       prevNoseCoords = nose_point
       
    dir, offset = movDirection(nose_point, prevNoseCoords)
    cv2.putText(image, dir.upper(), (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, RED_COLOR, 2)
    #print(offset[0],offset[1])
    
    pyautogui.move(-(offset[0]*4), offset[1]*5)
    
    prevNoseCoords = nose_point
    #Code end for mouse pointer
    

    #Checking if smile mode on and if ON change the 
    if smile_dist > SMILE_THRESH:
        SMILE_COUNTER += 1
        
        if SMILE_COUNTER > SMILE_CONSECUTIVE_FRAMES:
            print('SMILE Detected')
            cv2.putText(image,'Smile Detected',(50,90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, RED_COLOR, 2)
            SCROLL_MODE = not SCROLL_MODE  
            SMILE_COUNTER = 0
    else:
        SMILE_COUNTER = 0

        
    if SCROLL_MODE:
                if dir == 'Up' or dir == 'Left-Up' or dir == 'Right-Up':
                    pyautogui.scroll(40)
                elif dir == 'Down' or dir == 'Left-Down' or dir == 'Right-Down':
                    pyautogui.scroll(-40)
                    
                    
    print(mar)               
    if mar > MOUTH_AR_THRESH:
        MOUTH_COUNTER += 1
        
        if MOUTH_COUNTER > MOUTH_AR_CONSECUTIVE_FRAMES:
            print('LeftClick')
            cv2.putText(image,'Mouth Open',(50,90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, RED_COLOR, 2)
            pyautogui.click()
            MOUTH_COUNTER = 0
    else:
        MOUTH_COUNTER = 0
        
    
    #print(ear)
    if ear < EYE_AR_THRESH:
        EYE_COUNTER += 1
        
        if EYE_COUNTER > EYE_AR_CONSEC_FRAMES:
            print('RightClick')
            cv2.putText(image,'BLINK',(50,90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, RED_COLOR, 2)
            pyautogui.click(button='right')
            EYE_COUNTER = 0
    else:
        EYE_COUNTER = 0
        
    # Draw on our image, all the finded cordinate points (x,y) 
    for (x, y) in shape:
        cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
           
    isFirstFrame = False
    # Show the image
    windowName = "Output"
    cv2.namedWindow(windowName)        # Create a named window
    cv2.moveWindow(windowName, 0,0)
    cv2.imshow(windowName, image)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
    
    time2 = time.time()
    #print(time2-time1)
cv2.destroyAllWindows()
cap.release()

0.10618030373897001
0.11099976912587382
0.08209939560583421
0.07699974369619363
0.09535057164992713
0.08685878296558368
0.08256011246883166
0.0899432647867242
0.06009412171179704
0.08171153838862723
0.07576148583556337
0.062308917838007544
0.04025909778455144
0.025426944184368024
0.09060270894817386
0.10572461997582026
0.10921627957665847
0.10214073967525991
0.09449217975345012
0.06999650026247813
0.06070493954516134
0.0920814661726662
0.061296210596789844
0.07464908473055877
0.08386987215605593
0.08925476035771107
0.07928311769295797
0.06565380207197476
0.07505479216937597
0.06431002665193589
0.06327607090126869
0.07873302441563805
0.07364627202316579
0.06587038668783715
0.08421636509074377
0.07349544166706488
0.07428723170282896
0.06458814886803348
0.07971213721331381
0.12047953437008548
0.0684172623065265
0.06985613702374244
0.06988843297416114
0.09275562736337298
0.07131701338904727
0.15624152358477147
0.13869022413698892
0.11004552514255797
0.11918318918325643
0.11386138613861387
